In [ ]:
1.200个数据的代码

In [2]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta

# 初始化Faker库（用于生成随机时间）
fake = Faker()

# 预设参数
DATA_SIZE = 200  # 数据量
START_DATE = datetime(2025, 5, 1)  # 评论时间范围起始日
END_DATE = datetime(2025, 5, 19)   # 评论时间范围结束日

# 预设情感倾向及对应评分范围
sentiment_config = {
    '正面': {'prob': 0.4, 'score_range': (4, 5)},
    '中性': {'prob': 0.3, 'score_range': (3, 3)},
    '负面': {'prob': 0.3, 'score_range': (1, 2)}
}

# 预设品牌关键词（可根据需求扩展）
brand_keywords = [
    '质量', '服务', '物流', '设计', '价格', 
    '功能', '体验', '包装', '性价比', '售后'
]

# 预设评论模板（根据情感倾向和关键词组合生成）
comment_templates = {
    '正面': [
        "{}非常好，{}很贴心，下次还会回购！",
        "用了一周，{}超出预期，{}到位，必须好评！",
        "{}符合描述，{}快速，整体体验很棒！",
        "{}精致，{}合理，这个价位买到很值！"
    ],
    '中性': [
        "{}中规中矩，{}一般，没有特别惊喜。",
        "{}正常水平，{}速度还行，凑合用吧。",
        "{}和图片差不多，{}没太大亮点，就这样。"
    ],
    '负面': [
        "{}差，{}慢，问题一直没解决，失望！",
        "收到货{}破损，{}态度差，申请退货还麻烦！",
        "{}和描述不符，{}不专业，再也不会买了！",
        "{}刺鼻，{}推卸责任，这质量对得起价格吗？"
    ]
}

def generate_comment(sentiment, keywords):
    """根据情感倾向和关键词生成评论内容"""
    template = random.choice(comment_templates[sentiment])
    # 确保关键词数量匹配模板占位符（这里模板用2个占位符）
    k1 = random.choice(keywords)
    k2 = random.choice([k for k in keywords if k != k1] + [k1])  # 允许重复关键词
    return template.format(k1, k2)

def generate_data():
    """生成模拟评论数据"""
    data = []
    for idx in range(1, DATA_SIZE + 1):
        # 随机选择情感倾向（根据预设概率）
        sentiment = random.choices(
            list(sentiment_config.keys()),
            weights=[v['prob'] for v in sentiment_config.values()]
        )[0]
        
        # 根据情感倾向确定评分
        score_min, score_max = sentiment_config[sentiment]['score_range']
        score = random.randint(score_min, score_max)
        
        # 随机生成发布时间（在指定日期范围内）
        delta = (END_DATE - START_DATE).days
        random_days = random.randint(0, delta)
        post_date = (START_DATE + timedelta(days=random_days)).strftime("%Y-%m-%d")
        
        # 随机选择1-2个品牌关键词
        keywords = random.sample(brand_keywords, k=random.randint(1, 2))
        
        # 生成评论内容
        comment = generate_comment(sentiment, keywords)
        
        data.append({
            '评论ID': f"{idx:04d}",  # 格式化为4位数字（如0001）
            '评论内容': comment,
            '情感倾向': sentiment,
            '评分（1-5星）': score,
            '发布时间': post_date,
            '品牌关键词': '、'.join(keywords)
        })
    return data

if __name__ == "__main__":
    try:
        # 生成数据
        comment_data = generate_data()
        # 转换为DataFrame
        df = pd.DataFrame(comment_data)
        # 按发布时间排序（可选）
        df = df.sort_values(by='发布时间')
        # 导出为CSV文件
        df.to_csv('brand_comments.csv', index=False, encoding='utf-8-sig')
        print(f"成功生成 {DATA_SIZE} 条评论数据，已保存为 brand_comments.csv")
    except Exception as e:
        print(f"生成数据时发生错误: {str(e)}")


成功生成 200 条评论数据，已保存为 brand_comments.csv


In [ ]:
对数据进行预处理

In [3]:
import pandas as pd
import re
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

def load_data(file_path):
    """加载数据并检查基础信息"""
    df = pd.read_csv(file_path, encoding='utf-8-sig')
    print("===== 数据概览 =====")
    print(f"数据量: {df.shape[0]} 条")
    print(f"缺失值统计:\n{df.isnull().sum()}")
    print(f"情感倾向分布:\n{df['情感倾向'].value_counts()}")
    return df

def clean_text(text):
    """清洗评论内容（去除特殊符号、多余空格）"""
    # 去除非中文字符（保留字母、数字、基本标点）
    cleaned = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9，。！？：；“”（）]', ' ', text)
    # 合并连续空格
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned

def preprocess_data(df):
    """核心预处理函数"""
    # 1. 清洗评论内容
    df['评论内容_clean'] = df['评论内容'].apply(clean_text)
    
    # 2. 情感倾向编码（正面=1，中性=0，负面=-1）
    le = LabelEncoder()
    df['情感编码'] = le.fit_transform(df['情感倾向'])
    # 调整编码映射（原LabelEncoder默认按字母排序，这里手动修正为合理顺序）
    df['情感编码'] = df['情感倾向'].map({'正面': 1, '中性': 0, '负面': -1})
    
    # 3. 时间特征提取
    df['发布时间'] = pd.to_datetime(df['发布时间'])  # 转换为datetime类型
    df['发布月份'] = df['发布时间'].dt.month  # 月份（本例固定为5月）
    df['发布星期'] = df['发布时间'].dt.dayofweek  # 星期几（0=周一，6=周日）
    df['发布日'] = df['发布时间'].dt.day  # 日期（1-19日）
    
    # 4. 评论长度特征（字数）
    df['评论字数'] = df['评论内容_clean'].apply(lambda x: len(x))
    
    # 5. 关键词拆分（转为列表，方便后续分析）
    df['品牌关键词列表'] = df['品牌关键词'].apply(lambda x: x.split('、'))
    
    # 6. 评分验证（确保在1-5范围内）
    if not df['评分（1-5星）'].between(1, 5).all():
        print("警告：发现异常评分值，已修正！")
        df['评分（1-5星）'] = df['评分（1-5星）'].clip(1, 5)
    
    # 7. 去重（根据评论内容和发布时间）
    df = df.drop_duplicates(subset=['评论内容_clean', '发布时间'], keep='first')
    
    return df

if __name__ == "__main__":
    try:
        # 加载原始数据（请确保文件路径正确）
        raw_df = load_data('brand_comments.csv')
        
        # 执行预处理
        processed_df = preprocess_data(raw_df)
        
        # 保存预处理后的数据
        processed_df.to_csv('processed_comments.csv', index=False, encoding='utf-8-sig')
        print("\n===== 预处理完成 =====")
        print(f"预处理后数据量: {processed_df.shape[0]} 条")
        print("预处理结果已保存为 processed_comments.csv")
        print("关键新增列: 评论内容_clean、情感编码、发布星期、评论字数、品牌关键词列表")
        
    except FileNotFoundError:
        print("错误：未找到原始数据文件 'brand_comments.csv'，请先运行数据生成脚本！")
    except Exception as e:
        print(f"预处理过程中发生错误: {str(e)}")


===== 数据概览 =====
数据量: 200 条
缺失值统计:
评论ID        0
评论内容        0
情感倾向        0
评分（1-5星）    0
发布时间        0
品牌关键词       0
dtype: int64
情感倾向分布:
情感倾向
正面    86
中性    59
负面    55
Name: count, dtype: int64

===== 预处理完成 =====
预处理后数据量: 198 条
预处理结果已保存为 processed_comments.csv
关键新增列: 评论内容_clean、情感编码、发布星期、评论字数、品牌关键词列表
